# Sistemas multiagente
## Planificador de caminhos multiagente

### Pressupostos 
Vamos adoptar uma particular abordagem ao universo multiagente e seus problemas específicos.
Essa abordagem caracteriza-se por:

-  **Objectivo comum; agentes cooperantes**: Vamos considerar a existência de vários agentes, cada um dos quais tem o seu objectivo próprio. Podemos, no entanto, ver esta situação como havendo um objectivo comum de todos os agentes que consiste em que cada um deles consiga atingir o seu objectivo. 
Vamos portanto considerar que os agentes são inerentemente cooperantes, no sentido em que não pretendem impedir que os outros atinjam o seu objectivo.


- **Coordenação evita perturbação entre agentes**:
Cada agente planeia as suas acções. Apesar de os agentes serem cooperantes, as suas acções podem interferir com as acções de outros, perturbando-as, pelo que será necessário coordenar os seus planos de modo a evitar conflitos.

<img src="robotConflict.png" width="300">

- **sincronização perfeita**:
Há um relógio global; As ações de cada um dos agentes são iniciadas no mesmo instante temporal, 
são simultâneas e
demoram todas o mesmo tempo. 
Esta forma de sincronização é irrealista (um dos agentes pode ter que esperar) mas tem uma semântica simples.


- **agentes fracamente acoplados**:
começa por se tratar os agentes como se fossem independentes e depois repara-se o plano para as interações (coordenação)


A abordagem ao planeamento multiagente que vamos adoptar é conhecida por **$MAPF$**

### **Multiagent path finder (*MAPF*)**:
Segundo Stern et al. 2019 (http://arxiv.org/abs/1906.08291), o problema de encontrar um caminho multiagente (*multiagent path finder*, ou *MAPF*) é o de encontrar caminhos para múltiplos agentes, quando a restrição chave é que os agentes possam seguir concorrentemente esses caminhos sem colidir uns com os outros. As aplicações de *MAPF* incluem armazéns automatizados e veículos autónomos.


### *MAPF* Clássico
A descrição de um problema MAPF clássico com $k$ agentes é um tuplo $<Graph, start, target>$ onde $Graph = (V, A)$ é um grafo não direccionado, $start: [1, ..., k] \rightarrow V$ indica para cada agente o seu vértice origem e $target: [1, ..., k] \rightarrow V$ indica para cada agente o seu vértice alvo. 

Assume-se que o tempo é discretizado e em cada passo de tempo cada agente está situado em um dos vértices do grafo e pode realizar uma única acção. 

Uma acção é uma função $a: V \rightarrow V$ tal que $a(v) = v'$ significa que se o agente está no vértice $v$ e executa a acção $a$ então estará no vértice $v'$ no próximo passo de tempo. 

Cada agente tem dois tipos de acção, $wait$ and $move$. Uma acção $wait$ significa que o agente fica no mesmo vértice mais um passo de tempo. Uma acção $move$ quer dizer que o agente se move do seu vértice corrente $v$ para um vértice adjacente $v'$ no grafo (ou seja, $(v, v') \in A$).

Para uma sequência de acções $\pi = (a_1, ..., a_n)$ e um agente $i$, denota-se $\pi_i[x]$ a localização do agente após executar as primeiras $x$ acções de $\pi$, começando pelo vértice origem do agente,  $start(i)$. Ou seja, $\pi_i[x]= a_x(a_{x-1}(...(a_1(start(i))))$. 

Uma sequência de acções $\pi$ é um **plano para o agente $i$** se e só se executar esta sequência de acções em $start(i)$ resulta em estar em $target(i)$, ou seja, se $\pi_i[|\pi|] = target(i)$. 

Uma **solução** é um conjunto de $k$ planos, um para cada agente.




### Conflitos num problema *MAPF*
Para resolver um problema *MAPF* é útil a noção de **conflito** durante o planeamento. 

Uma solução para um problema *MAPF* é chamada **válida** se, para quaisquer agentes i e j,  não existe nenhum conflito entre o plano para i e o plano para j. 

Embora a noção de conflito dependa do ambiente específico considerado, podem listar-se vários tipos de conflitos entre planos. Vejamos alguns conflitos comuns, ilustrados nas seguintes imagens:

<img src="conflitosV2.png" alt="Drawing" style="width: 800px;"/>

Como podemos caracterizar cada um desses tipos de conflitos?

Sejam $\pi_i$ e $\pi_j$ dois planos para, respectivamente, um agente i e um agente j.

1. **Conflito de arco**: dois agentes estão planeados para percorrer o mesmo arco no mesmo passo de tempo na mesma direcção, ou seja, existe conflito de arco entre $\pi_i$ e $\pi_j$ se e só se existe um passo de tempo $x$ tal que $\pi_i[x]=\pi_j[x]$ e $\pi_i[x+1]=\pi_j[x+1]$.


2. **Conflito de vértice**: dois agentes estão planeados para ocupar o mesmo vértice no mesmo passo de tempo, ou seja, existe conflito de vértice entre $\pi_i$ e $\pi_j$ se e só se existe um passo de tempo $x$ tal que $\pi_i[x]=\pi_j[x]$.


3. **Conflito de seguimento**: um agente está planeado para ocupar um vértice que estava ocupado por outro agente no passo de tempo anterior, ou seja, existe conflito de seguimento entre $\pi_i$ e $\pi_j$ se e só se existe um passo de tempo $x$ tal que $\pi_i[x+1]=\pi_j[x]$.


4. **Conflito de ciclo entre os planos  $\pi_i, \pi_{i+1}, ... \pi_j$**: no mesmo passo de tempo cada agente move-se para um vértice que estava previamente ocupado por outro agente, formanto um padrão de "ciclo rotativo", ou seja, existe conflito de ciclo entre os planos $\pi_i, \pi_{i+1}, ... \pi_j$ se e só se existe um passo de tempo $x$ tal que $\pi_i[x+1]=\pi_{i+1}[x]$ e $\pi_{i+1}[x+1]=\pi_{i+2}[x]$ e ... e $\pi_{j-1}[x+1]=\pi_j[x]$ e $\pi_j[x+1]=\pi_i[x]$.


5. **Conflito de troca**: dois agentes estão planeados para trocar de posição num único passo de tempo, ou seja, existe conflito de troca entre $\pi_i$ e $\pi_j$ se e só se existe um passo de tempo $x$ tal que $\pi_i[x+1]=\pi_j[x]$ e $\pi_j[x+1]=\pi_i[x]$.


Num determinado problema de *MAPF* podemos estar dispostos a admitir algumas situações que identificámos  acima como um tipo de conflito, mas não admitir outras. 
Temos no entanto que estar consciente de que certos tipos de conflitos  implicam outros. Por exemplo:
* Proibir conflitos de vértice implica proibir conflitos de arco.
* Proibir conflitos de seguimento implica proibir conflitos de troca

Para definir com precisão um problema *MAPF* é preciso indicar que tipos de conflitos vamos proibir.

### Exemplo simples num mundo com grelha 4x4 e dois agentes

Começamos por um exemplo simples. O ficheiro ```mundo44.py``` apresenta um mundo com uma grelha de 4x4, dois agentes e dois objectivos que podem facilmente não gerar nenhum conflito.

*  Nota: O código que vamos usar recorre ao módulo tkinter (interface python para Tcl/tk) que poderá ter que instalar. 


In [1]:
### carregamos as ferramentas de visualização e definiçao do mundo
from gworld import *
from visualize import *


In [2]:
### Definimos um mundo simples
### O ficheiro mundo44.py tem exemplos de vários mundos e problemas 
 
w = GridWorld(4,4)    # as coordenadas da grelha são inteiros em [0,3]
w.add_rocks([(2,2),(3,3)])
w.add_agents([(0,0,2,1), (0,1,1,3)]) # para cada agente é dada a sua posição origem (start) e alvo (target)
agents = list(w.get_agents())
print(agents)

[(0, 0, 2, 1), (0, 1, 1, 3)]
[1, 2]


Ou seja, este mundo tem dois agentes, o primeiro começa em (0,0) e tem como objectivo ir para (2,1) e o segundo começa em (0,1) e tem como objectivo ir para (1,3).

O código seguinte abre uma janela "tk" com uma representação desse mundo-grelha, onde os agentes são representados por círculos coloridos e os seus objectivos por células cujo bordo está colorido com a cor do agente em causa. As pedras são as células a cinzento.

ATENÇÃO que a janela pode ficar por trás de outra, procurem-na que ela está por aí (ajuda se fecharem todas as janelas que não estão a usar e se a janela deste notebook não ocupar o écran todo!!!).

In [3]:
### Vejamos
vis = Visualize(w)
vis.draw_world()
vis.draw_agents()
vis.canvas.pack()
vis.canvas.update()

Cada agente pode movimentar-se uma posição na horizontal ou na vertical, desde que a célula para a qual se movimenta não esteja ocupada por uma pedra.

Isto é, tendo em conta a caracterização de um problema *MAPF* atrás indicada, o grafo em causa teria, a partir do vértice/célula ```(2,1)```, apenas arcos para os seguintes vértices:

```
(2,0)  # movimento para a esquerda
(1,1)  # movimento para cima
(3,1)  # movimento para baixo
```


O estado de cada agente vai ser representado por um tuplo  tempo-espaço ```(t, a, b)``` onde o ```t``` é a etiqueta temporal, e ```(a,b)``` são as coordenadas onde o agente está nesse instante ```t```.

Por exemplo, o agente 2 (agente azul) encontra-se no início no estado```(0, 0, 1)```, podendo passar no instante seguinte (t = 1) para um dos seguintes estados:
```
(1,0,1)   # pela acção wait
(1,0,0)   # por uma acção move, para a esquerda
(1,0,2)   # por uma acção move, para a direita
(1,1,1)   # por uma acção move, para baixo
```
As 2 primeiras alternativas podem gerar conflitos de vértice 
com as acções possíveis do outro agente, o agente 1 ou agente verde

## Encontrar uma solução para MAPF

Para encontrar uma solução para um problema *MAPF* vamos usar o algoritmo de procura informada **A***, independentemente para cada um dos agentes, procurando evitar conflitos entre os planos dos diversos agentes através do uso de uma tabela de reservas de estados, isto é, de tuplos tempo-espaço 

### Método centralizado num mundo em grelha
1. Encontrar caminhos independentes para cada agente sem considerar os outros agentes: Usar A\* tempo-espaço para procura de baixo nível
1. Fazer reservas numa tabela de reservas tempo-espaço
1. Verificar na tabela se todos os caminhos estão livres de conflitos com os outros agentes
1. Quando se encontra um conflito, adicionar uma restrição no planeador de baixo nível e replanear.


- Para evitar que os agentes passem uns através dos outros, cada agente faz múltiplas reservas na tabela de reservas
 - Isto por vezes causa um atraso considerável, com um ou mais agentes à espera na sua localização, mesmo que um caminho esteja claramente disponível
 - Para evitar isto, é preciso adicionar diferentes tipos de restrições: para colisões de vértices e para colisões de arcos
 
Link para vídeo: https://youtu.be/b5KMm729b_4



Voltando ao nosso exemplo na grelha 4x4 com os agentes 1 (verde) e 2 (azul), vamos agora procurar uma solução, isto é, um conjunto de 2 planos, um para cada agente, que não gerem conflitos.

Para esse fim, vamos importar o módulo cbsearchPlus que, por sua vez, importa o astar (algoritmo A* num mundo de grelha usando a distância de manhattan como heurística) e o m_astar (variante para ter em conta restrições)

In [4]:
### Importar algoritmo de procura
import cbsearchPlus as cbs
conflict = False
path_maxlen = 0
constraints = []


A função search do módulo cbsearchPlus tem a seguinte assinatura:

```search(agents, world, verbose = False, step_by_step = False)```

e retorna um triplo 

```(path_seq, pathcost, conflicts)```

onde cada um dos elementos do triplo é um dicionário cujas chaves correspondem ao número de um agente. 

Vamos primeiro obter uma solução com os valores *default* para ```verbose``` e ```step_by_step```.



É apresentado para cada agente: o custo do plano (*pathcost* que representa o número de movimentos) e a sequência de estados (*path_seq*)

In [5]:
### Encontrar os caminhos
path_seq, pathcost, conflicts_db = cbs.search(agents, w)
print()
print("RESULTADOS DA PROCURA")
print("PLANOS E CONFLITOS RESOLVIDOS")
for agent in agents:
    print('Agente ', agent,":")
    print(' custo:',pathcost[agent])
    print(' Estados:')
    for estado in path_seq[agent]:
        print('  ', estado)
    print("  Conflitos que foram resolvidos:")
    if agent in conflicts_db:
        print('  ', conflicts_db[agent])
    else:
        print(None)


RESULTADOS DA PROCURA
PLANOS E CONFLITOS RESOLVIDOS
Agente  1 :
 custo: 3
 Estados:
   (0, 0, 0)
   (1, 0, 1)
   (2, 1, 1)
   (3, 2, 1)
   (4, 2, 1)
   (5, 2, 1)
   (6, 2, 1)
   (7, 2, 1)
   (8, 2, 1)
   (9, 2, 1)
   (10, 2, 1)
   (11, 2, 1)
   (12, 2, 1)
   (13, 2, 1)
   (14, 2, 1)
   (15, 2, 1)
   (16, 2, 1)
   (17, 2, 1)
   (18, 2, 1)
   (19, 2, 1)
   (20, 2, 1)
   (21, 2, 1)
   (22, 2, 1)
   (23, 2, 1)
   (24, 2, 1)
   (25, 2, 1)
  Conflitos que foram resolvidos:
   set()
Agente  2 :
 custo: 3
 Estados:
   (0, 0, 1)
   (1, 0, 2)
   (2, 0, 3)
   (3, 1, 3)
   (4, 1, 3)
   (5, 1, 3)
   (6, 1, 3)
   (7, 1, 3)
   (8, 1, 3)
   (9, 1, 3)
   (10, 1, 3)
   (11, 1, 3)
   (12, 1, 3)
   (13, 1, 3)
   (14, 1, 3)
   (15, 1, 3)
   (16, 1, 3)
   (17, 1, 3)
   (18, 1, 3)
   (19, 1, 3)
   (20, 1, 3)
   (21, 1, 3)
   (22, 1, 3)
   (23, 1, 3)
   (24, 1, 3)
   (25, 1, 3)
  Conflitos que foram resolvidos:
   {(1, 0, 1)}


Porque é que os planos se estendem, após ser atingido o objetivo do agente? 

Isso está a ser controlado pela constante SOMETIME definida no ficheiro macros.py, um dos ficheiros importados por cbsearchPlus.


In [6]:
print("Tempo limite = ", SOMETIME) #SOMETIME está definida em macros.py

Tempo limite =  25


#### Hipóteses sobre o comportamento dos agentes ao atingir o alvo

Como os agentes podem chegar aos seus alvos em diferentes alturas, temos de especificar o seu comportamento entre o passo de tempo no qual chegou ao seu alvo e o passo de tempo quando o último agente chegou ao seu alvo. Duas suposições comuns são estas:
* Fica no alvo: Ficar à espera no seu alvo pode causar um conflito de vértice com algum plano que passe por esse vértice depois do agente ter chegado. Sob a hipótese fica-no-alvo, um par de planos $\pi_i$ e $\pi_j$ terá um conflito de vértice se existe um passo de tempo $t \gt |\pi_i|$ tal que $\pi_j[t] = \pi_i[|\pi_i|]$. 
* Desaparece no alvo: Sob esta hipótese não haverá quaisquer conflitos depois do passo de tempo no qual o agente correspondente alcança o seu alvo (por exemplo, um carro que ao chegar ao seu destino é estacionado numa garagem).

A hipótese adoptada nesta implementação é fica no alvo.





###  Exemplo simples em maior detalhe 

Vamos agora ver a procura da solução na versão ```verbose = True``` e ```step_by_step = True```.

De vez em quando é emitido algum output, com informação sobre a situação da procura. Vejam o output e carreguem na tecla  <enter\> para continuar.

* IMPORTANTE: por vezes a solução não é perfeita e gera excepções. Nesse caso, repetir o processo de procura normalmente é o suficiente. Há um "shuffle" dos agentes que permite encontrar uma solução melhor.

In [8]:
path_seq, pathcost, conflicts_db = cbs.search(agents, w, True, True)
print()
print("RESULTADOS DA PROCURA")
print("PLANOS E CONFLITOS RESOLVIDOS")
for agent in agents:
    print('Agente ', agent,":")
    print(' custo:',pathcost[agent])
    print(' Estados:')
    for estado in path_seq[agent]:
        print('  ', estado)
    print("  Conflitos que foram resolvidos:")
    if agent in conflicts_db:
        print('  ', conflicts_db[agent])
    else:
        print(None)

Current plans:
 for agent  1 :  [(0, 0, 0), (1, 0, 1), (2, 1, 1), (3, 2, 1)]
 for agent  2 :  [(0, 0, 1), (1, 0, 2), (2, 1, 2), (3, 1, 3)]
Press enter to continue ...

for agent  2 :
 Start: (0, 0, 1)  Target: (25, 1, 3)  Constraints:  {(1, 0, 1)}
 New Path:  [(0, 0, 1), (1, 0, 2), (2, 0, 3), (3, 1, 3), (4, 1, 3), (5, 1, 3), (6, 1, 3), (7, 1, 3), (8, 1, 3), (9, 1, 3), (10, 1, 3), (11, 1, 3), (12, 1, 3), (13, 1, 3), (14, 1, 3), (15, 1, 3), (16, 1, 3), (17, 1, 3), (18, 1, 3), (19, 1, 3), (20, 1, 3), (21, 1, 3), (22, 1, 3), (23, 1, 3), (24, 1, 3), (25, 1, 3)]
Press enter to continue ...

Loop break!
Press enter to continue ...


RESULTADOS DA PROCURA
PLANOS E CONFLITOS RESOLVIDOS
Agente  1 :
 custo: 3
 Estados:
   (0, 0, 0)
   (1, 0, 1)
   (2, 1, 1)
   (3, 2, 1)
   (4, 2, 1)
   (5, 2, 1)
   (6, 2, 1)
   (7, 2, 1)
   (8, 2, 1)
   (9, 2, 1)
   (10, 2, 1)
   (11, 2, 1)
   (12, 2, 1)
   (13, 2, 1)
   (14, 2, 1)
   (15, 2, 1)
   (16, 2, 1)
   (17, 2, 1)
   (18, 2, 1)
   (19, 2, 1)
   (20, 2, 1

In [9]:
### Vamos agora ver a solução representada no mundo w, na janela tk
## inicialização 
action_seq = dict()
path_maxlen = 0

for agent in agents:
    path_len = len(path_seq[agent])
    path_maxlen = path_len if (path_len > path_maxlen) else path_maxlen
    action_seq[agent] = w.path_to_action(agent, path_seq[agent])

for step in range(path_maxlen):
    for agent in agents:
        if( action_seq[agent] ):
            action = action_seq[agent].pop(0)
            w.agent_action(agent, action)
            vis.canvas.update()
            vis.canvas.after(200)
    vis.canvas.update()
    


Exception: Cell is not unoccupied! : (0,0) --> 0

### Exemplo numa grelha 6x10 

Vejam agora este mundo um pouco mais complicado. 

Apesar da grelha ser maior, nomeadamente no número de linhas, há muitas pedras/obstáculos que impedem os dois agentes de se deslocarem para linhas com índice > 1.

Os planos dos agentes têm também mais possibilidade de entrar em conflito. Nomeadamente, o agente 1 (agente verde) terá que passar por cima do agente 2 (agente azul) para conseguir atingir o seu objectivo.

In [10]:
w1 = GridWorld(6,10)
w1.add_rocks( [ (1,0),(1,1),(1,2),(1,3),(1,4),(2,5),(1,6),(1,7),(1,8),(1,9),(0,9) ] )
w1.add_agents( [ (0,0,0,8), (0,1,0,7) ] )

vis = Visualize(w1)
vis.draw_world()
vis.draw_agents()
vis.canvas.pack()
vis.canvas.update()
agents = list(w1.get_agents())

### Preparar
conflict = False
path_maxlen = 0
constraints = []
path_seq = dict()


[(0, 0, 0, 8), (0, 1, 0, 7)]


In [12]:
### Encontrar os caminhos
### Encontrar os caminhos
path_seq, pathcost, conflicts_db = cbs.search(agents, w1)
print()
print("RESULTADOS DA PROCURA")
print("PLANOS E CONFLITOS RESOLVIDOS")
for agent in agents:
    print('Agente ', agent,":")
    print(' custo:',pathcost[agent])
    print(' Estados:')
    for estado in path_seq[agent]:
        print('  ', estado)
    print("  Conflitos que foram resolvidos:")
    if agent in conflicts_db:
        print('  ', conflicts_db[agent])
    else:
        print(None)


RESULTADOS DA PROCURA
PLANOS E CONFLITOS RESOLVIDOS
Agente  1 :
 custo: 8
 Estados:
   (0, 0, 0)
   (1, 0, 1)
   (2, 0, 2)
   (3, 0, 3)
   (4, 0, 4)
   (5, 0, 5)
   (6, 0, 6)
   (7, 0, 7)
   (8, 0, 8)
   (9, 0, 8)
   (10, 0, 8)
   (11, 0, 8)
   (12, 0, 8)
   (13, 0, 8)
   (14, 0, 8)
   (15, 0, 8)
   (16, 0, 8)
   (17, 0, 8)
   (18, 0, 8)
   (19, 0, 8)
   (20, 0, 8)
   (21, 0, 8)
   (22, 0, 8)
   (23, 0, 8)
   (24, 0, 8)
   (25, 0, 8)
  Conflitos que foram resolvidos:
   set()
Agente  2 :
 custo: 6
 Estados:
   (0, 0, 1)
   (1, 0, 2)
   (2, 0, 3)
   (3, 0, 4)
   (4, 0, 5)
   (5, 1, 5)
   (6, 1, 5)
   (7, 0, 5)
   (8, 0, 6)
   (9, 0, 7)
   (10, 0, 7)
   (11, 0, 7)
   (12, 0, 7)
   (13, 0, 7)
   (14, 0, 7)
   (15, 0, 7)
   (16, 0, 7)
   (17, 0, 7)
   (18, 0, 7)
   (19, 0, 7)
   (20, 0, 7)
   (21, 0, 7)
   (22, 0, 7)
   (23, 0, 7)
   (24, 0, 7)
   (25, 0, 7)
  Conflitos que foram resolvidos:
   {(1, 0, 1), (2, 0, 2), (5, 0, 5), (4, 0, 4), (3, 0, 3), (6, 0, 6), (7, 0, 7), (8, 0, 8), (7, 0,

Vamos ver a solução.

In [13]:
### Vamos agora ver a solução no canvas
action_seq = dict()

for agent in agents:
    path_len = len(path_seq[agent])
    path_maxlen = path_len if (path_len > path_maxlen) else path_maxlen
    action_seq[agent] = w1.path_to_action(agent, path_seq[agent])

for step in range(path_maxlen):
    for agent in agents:
        if( action_seq[agent] ):
            action = action_seq[agent].pop(0)
            w1.agent_action(agent, action)
            vis.canvas.update()
            vis.canvas.after(200)
    vis.canvas.update()


Exception: Cell is not unoccupied! : (0,0) --> 0

### Exercícios

Vejam os vários mundos definidos abaixo (ficheiro mundo44.py) e executem o algoritmo de planeamento. Há vários cenários comentados, para irem descomentando à vez. 

Reparem que mal o planeador encontra um conflito expande os planos para SOMETIME passos (variável ```SOMETIME``` em ```macros.py```), o que conduz a planos que por vezes têm muita redundância. Os planos estão guardados ```path_seq```, um dicionário que associa a cada agente o seu plano. Definam uma função ```limpa_redundancia``` que dado um dicionário de plano (```path_seq```) retire os passos finais, quando o agente já não se move mais.

In [12]:
from gworld import *
from visualize import *
import m_astar
import cbsearchPlus as cbs


## mundo simples para debugging
## e exemplos
a = GridWorld(4,4)
a.add_rocks([(1,2),(2,2),(3,3)])
a.add_agents([(0,0,2,1), (0,1,1,3)])

## Go around block. Wait aside for agent1 to pass
## Takes too long. Need better conflict handling
##a = GridWorld(6,10)
##a.add_rocks( [ (2,1),(1,2),(1,3),(1,4),(3,1),(2,3),(3,3),(3,4) ] )
##a.add_agents( [ (1,0,3,2), (1,1,2,2) ] )

## 2 agents. Narrow path with a open slot on the wall
## Waits too long. Need better conflict handling
##a = GridWorld(6,10)
##a.add_rocks( [ (1,0),(1,1),(1,2),(1,3),(1,4),(2,5),(1,6),(1,7),(1,8),(1,9),(0,9) ] )
##a.add_agents( [ (0,0,0,8), (0,1,0,7) ] )

## 3 agents. Few rocks. More space to swerve around
#a = GridWorld(6,10)
#a.add_rocks( [ (4,0),(4,1),(4,2),(1,7),(1,8),(1,9) ] )
#a.add_agents( [ (0,7,5,1), (5,3,0,9), (0,3,5,9) ] )

## 3 agents. Single passable block
#a = GridWorld(6,10)
#a.add_rocks( [ (4,0),(4,1),(4,2),(4,3),(4,4),(3,4),(1,6),(1,7),(1,8),(1,9) ] )
#a.add_agents( [ (0,7,5,1), (5,3,0,9), (0,3,5,9) ] )

## 4 agents. Few rocks. More space to swerve around
## Need better conflict handling for an optimal path
# a = GridWorld(6,10)
# a.add_rocks( [ (4,0),(4,1),(4,2),(1,7),(1,8),(1,9) ] )
# a.add_agents( [ (0,7,5,1), (5,3,0,9), (0,3,5,9), (3,0,3,9) ] )


vis = Visualize(a)

vis.draw_world()
vis.draw_agents()

vis.canvas.pack()
vis.canvas.update()
vis.canvas.after(500)

agents = list(a.get_agents())

conflict = False
path_maxlen = 0
constraints = []

path_seq = dict()
path_seq, pathcost, conflicts_db  = cbs.search(agents, a)

### AQUI
### limpa_redundancia(path_seq)

something = input('Press 2 + <enter> to continue...')

action_seq = dict()
for agent in agents:
    path_len = len(path_seq[agent])
    path_maxlen = path_len if (path_len > path_maxlen) else path_maxlen
    action_seq[agent] = a.path_to_action(agent, path_seq[agent])

for step in range(path_maxlen):
    for agent in agents:
        # print 'ActSeq: ', agent, action_seq[agent]
        if( action_seq[agent] ):
            action = action_seq[agent].pop(0)
            a.agent_action(agent, action)
            vis.canvas.update()
            vis.canvas.after(150)
    vis.canvas.update()
    vis.canvas.after(500)

vis.canvas.update()
vis.canvas.after(500)


[(0, 0, 2, 1), (0, 1, 1, 3)]
Press 2 + <enter> to continue...


In [13]:
##### limpa_redundancia aqui
# def limpa_redundancia(path_seq):

#### um exemplo de path_seq a ser limpo

path1 = {1: [(0, 0, 0), (1, 1, 0), (2, 1, 1), (3, 2, 1), (4, 2, 1), (5, 2, 1), (6, 2, 1), 
             (7, 2, 1), (8, 2, 1), (9, 2, 1), (10, 2, 1), (11, 2, 1), (12, 2, 1), 
             (13, 2, 1), (14, 2, 1), (15, 2, 1), (16, 2, 1), (17, 2, 1), (18, 2, 1), 
             (19, 2, 1), (20, 2, 1), (21, 2, 1), (22, 2, 1), (23, 2, 1), (24, 2, 1), 
             (25, 2, 1)], 
         2: [(0, 0, 1), (1, 0, 2), (2, 0, 3), (3, 1, 3), (4, 1, 3), (5, 1, 3), (6, 1, 3), 
             (7, 1, 3), (8, 1, 3), (9, 1, 3), (10, 1, 3), (11, 1, 3), (12, 1, 3), (13, 1, 3), 
             (14, 1, 3), (15, 1, 3), (16, 1, 3), (17, 1, 3), (18, 1, 3), (19, 1, 3), 
             (20, 1, 3), (21, 1, 3), (22, 1, 3), (23, 1, 3), (24, 1, 3), (25, 1, 3)]}
